In [73]:
%reload_ext swagpy.jupyter
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

from sklearn.naive_bayes import GaussianNB
import tensorflow as tf

from swagpy.constants import Store, PROBSEVERE as PS


print(
    [
        tf.config.experimental.get_device_details(gpu)
        for gpu in tf.config.list_physical_devices("GPU")
    ]
)

2022-10-07 17:05:35.891129: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 17:05:36.565611: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-07 17:05:38.116524: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-10-07 17:05:38.116883: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

[{'compute_capability': (7, 5), 'device_name': 'NVIDIA GeForce RTX 2080 SUPER'}]


2022-10-07 17:05:40.343801: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-07 17:05:40.452763: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-07 17:05:40.452872: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-07 17:05:40.453224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
ps = xr.open_zarr(PS.store)
ps

<xarray.Dataset>
Dimensions:            (time: 76020)
Coordinates:
  * time               (time) datetime64[ns] 2022-09-01T00:00:40 ... 2022-09-...
Data variables: (12/28)
    CAPE_M10M30        (time) uint32 dask.array<chunksize=(38010,), meta=np.ndarray>
    EBSHEAR            (time) float32 dask.array<chunksize=(38010,), meta=np.ndarray>
    FLASH_DENSITY      (time) float32 dask.array<chunksize=(38010,), meta=np.ndarray>
    FLASH_RATE         (time) uint32 dask.array<chunksize=(38010,), meta=np.ndarray>
    ID                 (time) uint32 dask.array<chunksize=(38010,), meta=np.ndarray>
    LJA                (time) float32 dask.array<chunksize=(38010,), meta=np.ndarray>
    ...                 ...
    SIZE               (time) uint32 dask.array<chunksize=(38010,), meta=np.ndarray>
    SRH01KM            (time) uint32 dask.array<chunksize=(38010,), meta=np.ndarray>
    VIL_DENSITY        (time) float32 dask.array<chunksize=(38010,), meta=np.ndarray>
    WETBULB_0C_HGT     (time) float32 dask.array<chunksize=(38010,), meta=np.ndarray>
    X                  (time) float32 dask.array<chunksize=(38010,), meta=np.ndarray>
    Y                  (time) float32 dask.array<chunksize=(38010,), meta=np.ndarray>

In [164]:
import numpy as np
import pandas as pd

TOOO_SLOW = 100
idx = pd.IndexSlice


def make_gridspace(
    x1: float, y1: float, x2: float, y2: float, x_size: int = 972, y_size: int = 635
) -> pd.DataFrame:

    x = np.linspace(x1, x2, x_size, dtype=np.float32)
    y = np.linspace(y1, y2, y_size, dtype=np.float32)

    grid = (
        pd.DataFrame(
            columns=pd.Index(x, name="lon"),
            index=pd.Index(y, name="lat"),
        )
        .unstack("lat")
        .reset_index()
        .dropna(axis=1)
    )
    grid["water_vapor"] = np.random.randint(0, 255, size=len(grid))
    grid["long_wave_ir"] = np.random.randint(0, 255, size=len(grid))
    return grid.set_index(["lat", "lon"]).sort_index(level=["lat", "lon"])


def make_features(
    x1: int, y1: int, x2: int, y2: int, n_features: int = 76_020
) -> pd.DataFrame:
    xx1 = np.random.randint(x1, x2, n_features)
    xx2 = np.random.randint(x1, x2, n_features)
    yy1 = np.random.randint(y2, y1, n_features)
    yy2 = np.random.randint(y2, y1, n_features)
    data = {
        "minx": np.min([xx1, xx2], axis=0),
        "maxx": np.max([xx1, xx2], axis=0),
        "miny": np.min([yy1, yy2], axis=0),
        "maxy": np.max([yy1, yy2], axis=0),
    }
    return pd.DataFrame(data)


def min_diff(target: np.ndarray, values: np.ndarray) -> np.ndarray:
    diff = abs(target[:, np.newaxis] - values)
    index = np.argmin(diff, axis=0)
    return target[index]


def main():
    # setting up the gridspace
    xy_1 = -129, 54  # west, north
    xy_2 = -60, 20  # east, south
    x_size, y_size = 972, 635
    X1, Y1 = xy_1
    X2, Y2 = xy_2
    gridspace = make_gridspace(X1, Y1, X2, Y2, x_size=x_size, y_size=y_size)
    lat, lon = (gridspace.index.unique(crd).to_numpy() for crd in ("lat", "lon"))
    # FEATURES
    n_features = 76_020
    features = make_features(X1, Y1, X2, Y2, n_features=n_features)

    min_lon = min_diff(lon, features.minx.to_numpy())
    max_lon = min_diff(lon, features.maxx.to_numpy())
    min_lat = min_diff(lat, features.miny.to_numpy())
    max_lat = min_diff(lat, features.maxy.to_numpy())
    # How can this loop be vectorized
    result = [
        gridspace.loc[idx[x1:x2, y1:y2, :]]
        for x1, x2, y1, y2 in np.c_[min_lat, max_lat, min_lon, max_lon][:TOOO_SLOW]
    ]
    return result


# if __name__ == "__main__":
main()

[                       water_vapor  long_wave_ir
 lat       lon                                   
 24.022081 -127.010300          252           195
           -126.939240            7            16
           -126.868179          248            96
           -126.797119          252            38
           -126.726059           57            92
 ...                            ...           ...
 27.025236 -80.252319           198            62
           -80.181259           129           171
           -80.110199           211           142
           -80.039131           135           175
           -79.968071           187           124
 
 [37791 rows x 2 columns],
                       water_vapor  long_wave_ir
 lat       lon                                  
 41.987381 -92.972191           21           227
           -92.901131           33            61
           -92.830070           66           129
           -92.759010          117           230
           -92.687950      

In [166]:
xy_1 = -129, 54  # west, north
xy_2 = -60, 20  # east, south
x_size, y_size = 972, 635
X1, Y1 = xy_1
X2, Y2 = xy_2
gridspace = make_gridspace(X1, Y1, X2, Y2, x_size=x_size, y_size=y_size)
lat, lon = (gridspace.index.unique(crd).to_numpy() for crd in ("lat", "lon"))
# FEATURES
n_features = 76_020
features = make_features(X1, Y1, X2, Y2, n_features=n_features)

min_lon = min_diff(lon, features.minx.to_numpy())
max_lon = min_diff(lon, features.maxx.to_numpy())
min_lat = min_diff(lat, features.miny.to_numpy())
max_lat = min_diff(lat, features.maxy.to_numpy())
min_lat

array([46.00946426, 35.01577377, 32.01261902, ..., 24.02208138,
       33.99684525, 20.        ])

In [171]:
import gridspace

gridspace.fit_grid(min_lat.astype(np.float32))

array([46.009464, 35.015774, 32.01262 , ..., 24.022081, 33.996845,
       20.      ], dtype=float32)

In [139]:
f = f.sort_index(level=["lat", "lon"])#.loc[idx[54.000000:53.892746, :], :]

[
        f.loc[idx[x1:x2, y1:y2], :]
        for x1, x2, y1, y2 in np.c_[min_lat, max_lat, min_lon, max_lon][:2]
    ]

water_vapor  long_wave_ir
lat  lon                                   
20.0 -129.000000           84           162
     -128.928940          178           208
     -128.857880          234            21
     -128.786819          107           205
     -128.715759          176           126
...                       ...           ...
54.0 -60.284245            13             1
     -60.213181           177            85
     -60.142120           251           160
     -60.071060           144             3
     -60.000000             0           217

[617220 rows x 2 columns]

In [100]:
def index_of_min_diff(target: np.ndarray, values: np.ndarray) -> np.ndarray:
    diff = abs(target[:, np.newaxis] - values)
    index = np.argmin(diff, axis=0)
    return target[index]


lat, lon = np.unique(grid.lat), np.unique(grid.lon)

min_lon = index_of_min_diff(lon, features.minx.to_numpy(dtype=np.float32))
max_lon = index_of_min_diff(lon, features.maxx.to_numpy(dtype=np.float32))
min_lat = index_of_min_diff(lat, features.miny.to_numpy(dtype=np.float32))
max_lat = index_of_min_diff(lat, features.maxy.to_numpy(dtype=np.float32))

In [112]:
idx = pd.IndexSlice
np.c_[min_lat, max_lat]
TOOO_SLOW = 100
space = grid.set_index(["lat", "lon"]).unstack("lon")


[
    space.loc[x1:x2, idx[:, y1:y2]]
    for x1, x2, y1, y2 in np.c_[min_lat, max_lat, min_lon, max_lon][:TOOO_SLOW]
]

[          water_vapor                                                  \
 lon       -109.031929 -108.960869 -108.889801 -108.818741 -108.747681   
 lat                                                                     
 35.981071         207         152          39          54         224   
 36.034698         236         201         105         188          87   
 36.088329         212          19         171          94          12   
 36.141956         206          71         220         118          90   
 36.195583         171         180         193         221         208   
 ...               ...         ...         ...         ...         ...   
 42.791798          83          83         161         240         144   
 42.845425         181          65          89           2          61   
 42.899055          15          96          36          44         125   
 42.952682          48         220          11         109          90   
 43.006310         160         142    

In [71]:
def index_of_min_diff(target:np.ndarray, values:np.ndarray) ->np.ndarray:
    diff = abs(target[:, np.newaxis] - values)
    index = np.argmin(diff, axis=0)
    return index

lat, lon = np.unique(grid.lat), np.unique(grid.lon)

features["min_lon"] = index_of_min_diff(lon, features.minx.to_numpy(dtype=np.float32))
features["max_lon"] = index_of_min_diff(lon, features.maxx.to_numpy(dtype=np.float32))
features["min_lat"] = index_of_min_diff(lat, features.miny.to_numpy(dtype=np.float32))
features["max_lat"] = index_of_min_diff(lat, features.maxy.to_numpy(dtype=np.float32))
components =["min_lon","max_lon","min_lat","max_lat"]

features.set_index(components)#[components]#.stack(components)

minx  maxx  miny  maxy
min_lon max_lon min_lat max_lat                        
281     746     298     429      -109   -76    36    43
633     844     541     615       -84   -69    49    53
        957     317     410       -84   -61    37    42
183     619     261     597      -116   -85    34    52
535     718     261     354       -91   -78    34    39
...                               ...   ...   ...   ...
211     239     242     354      -114  -112    33    39
70      605     280     317      -124   -86    35    37
493     690     149     541       -94   -80    28    49
169     858     0       205      -117   -68    20    31
324     394     466     559      -106  -101    45    50

[76020 rows x 4 columns]

In [56]:
# .rename(columns={"min_lon":"lon",	'max_lon':"lon",	'min_lat':"lat",	'max_lat':"lat"})
DIMENSIONS = ["lat", "lon"] + list({"min_lon":"lon",	'max_lon':"lon",	'min_lat':"lat",	'max_lat':"lat"}.keys())
grid[]
# pd.concat([features,grid])
# grid.merge(features, how="outer", left_on=["lat", "lon"] , right_on=list({"min_lon":"lon",	'max_lon':"lon",	'min_lat':"lat",	'max_lat':"lat"}.keys()))
# print(features)
# grid.merge(features, how="outer", left_on=DIMENSIONS, right_on=DIMENSIONS)
# grid.merge(features, on=grid.lon)
# grid.lat.between([1,2,3], [2,3,4])

,lon,lat,water_vapor,long_wave_ir
0,-129.0,54.000000,183,61
1,-129.0,53.946373,56,18
2,-129.0,53.892746,61,212
3,-129.0,53.839115,136,222
4,-129.0,53.785488,190,247
...,...,...,...,...
617215,-60.0,20.214512,216,56
617216,-60.0,20.160883,105,177
617217,-60.0,20.107256,215,35
617218,-60.0,20.053627,77,131


In [31]:
idx = pd.IndexSlice
grid.set_index(["lon","lat"]).unstack("lat").loc[features.min_lon, idx[:, features.min_lat]]#.stack("lat") #.groupby(features.min_lon)#["water_vapor"].describe()

,water_vapor,long_wave_ir,water_vapor,long_wave_ir,water_vapor,long_wave_ir,water_vapor,long_wave_ir,water_vapor,long_wave_ir,...,water_vapor,long_wave_ir,water_vapor,long_wave_ir,water_vapor,long_wave_ir,water_vapor,long_wave_ir,water_vapor,long_wave_ir
lat,35.981071,35.981071,49.012619,49.012619,37.000000,37.000000,33.996845,33.996845,21.984226,21.984226,...,41.022083,41.022083,49.977917,49.977917,52.981071,52.981071,52.015774,52.015774,50.996845,50.996845
lon,,,,,,,,,,,,,,,,,,,,,
-109.031929,207,204,14,119,94,60,141,88,57,172,...,242,98,79,13,220,89,98,182,234,118
-84.018539,179,85,158,12,42,82,74,127,85,19,...,160,228,85,236,183,99,175,30,188,45
-84.018539,179,85,158,12,42,82,74,127,85,19,...,160,228,85,236,183,99,175,30,188,45
-115.995880,60,140,1,228,160,2,75,188,117,48,...,217,160,18,191,242,88,103,28,103,190
-90.982491,117,25,48,72,185,124,228,254,26,73,...,245,127,201,234,157,43,168,164,49,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-114.006180,130,64,161,66,119,14,233,243,102,64,...,117,69,243,113,232,8,89,187,234,28
-124.025749,0,107,240,144,187,140,239,198,68,163,...,166,67,75,203,204,242,174,252,112,70


In [17]:
# import
from typing import Callable
import dask.array as da
from functools import wraps

idx = pd.IndexSlice

def use_dask(func: Callable):
    @wraps(func)
    def wrapped(arr1:np.ndarray, arr2:np.ndarray):
        result: da.Array = func(da.from_array(arr1, chunks=76020//5), da.from_array(arr2))
        return result.compute()

    return wrapped








# @use_dask
# def f(arr1, arr2):
#     return arr1 - arr2
#     ...

# arr = da.from_array(gs.lon.to_numpy(dtype=np.float32)[:, np.newaxis], chunks=9000) - da.from_array(features.minx.to_numpy(dtype=np.float32))
# da.rechunk(arr,chunks=9000).compute()


# condition = (features["min_lon"].to_numpy()[:, np.newaxis] <= lon) & (
#     features["max_lon"].to_numpy()[:, np.newaxis] >= lon
# )

# da.from_array(gs.lon[:, np.newaxis])


# condition.any(axis=0).shape  # .all(axis=1).any()

# .to_numpy()[:, np.newaxis] < gs.lon.to_numpy()
# gs.set_index(
#     ["lon", "lat"]
# )  # .loc[idx[features["min_lon"] : features["max_lon"], :], :]

: 

: 

In [10]:
from swagpy.constants import MRMS_BOUNDS

W, E, S, N = MRMS_BOUNDS
ds = xr.open_zarr(Store.gmgsi)

lat, lon = ds["lat"].to_numpy(), ds["lon"].to_numpy()
lat = lat[(lat > S) & (lat < N)]  # [:, np.newaxis]
lon = lon[(lon > W) & (lon < E)]  # [:, np.newaxis]
ds = ds.sel({"lat": lat, "lon": lon}).drop(["GMGSI_SSR", "GMGSI_VIS"])
gmgsi = ds.to_dataframe().reset_index(["lat", "lon"])
np.unique(gmgsi["lon"])  # gmgsi.lon.shape

array([-129.9541626 , -129.88215637, -129.81015015, -129.73812866,
       -129.66612244, -129.59411621, -129.52210999, -129.45010376,
       -129.37809753, -129.30607605, -129.23406982, -129.1620636 ,
       -129.09005737, -129.01805115, -128.94602966, -128.87402344,
       -128.80201721, -128.73001099, -128.65800476, -128.58599854,
       -128.51397705, -128.44197083, -128.3699646 , -128.29795837,
       -128.22595215, -128.15393066, -128.08192444, -128.00991821,
       -127.93791199, -127.86590576, -127.79389191, -127.72188568,
       -127.64987183, -127.5778656 , -127.50585938, -127.43384552,
       -127.36183929, -127.28983307, -127.21781921, -127.14581299,
       -127.07380676, -127.00179291, -126.92978668, -126.85777283,
       -126.7857666 , -126.71376038, -126.64174652, -126.5697403 ,
       -126.49773407, -126.42572021, -126.35371399, -126.28170776,
       -126.20969391, -126.13768768, -126.06568146, -125.9936676 ,
       -125.92166138, -125.84964752, -125.7776413 , -125.70563

In [16]:
from typing import Literal
from numpy.typing import NDArray


def index_min(
    values: np.ndarray, axis: Literal["x", "y"]
) -> NDArray[np.unsignedinteger]:

    match axis:
        case "x":
            target = lat
        case "y":
            target = lon
        case _:
            raise Exception

    values -  target



In [4]:
from geopandas import GeoSeries
from geopandas.array import GeometryArray
from shapely.geometry import GeometryCollection, MultiPoint, MultiPolygon, box
import pygeos
import shapely.wkb
from geopandas._vectorized import from_wkb, from_wkt
import shapely.geometry


points = GeoSeries(pygeos.to_shapely(pygeos.points(gmgsi[["lon", "lat"]].to_numpy())))
print(points)

/tmp/ipykernel_14002/1446863516.py:10: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  points = GeoSeries(pygeos.to_shapely(pygeos.points(gmgsi[["lon", "lat"]].to_numpy())))


0          POINT (-129.95416 20.02966)
1          POINT (-129.88216 20.02966)
2          POINT (-129.81015 20.02966)
3          POINT (-129.73813 20.02966)
4          POINT (-129.66612 20.02966)
                      ...             
1234435     POINT (-60.32150 54.98364)
1234436     POINT (-60.24949 54.98364)
1234437     POINT (-60.17748 54.98364)
1234438     POINT (-60.10547 54.98364)
1234439     POINT (-60.03346 54.98364)
Length: 1234440, dtype: geometry


In [9]:
from geopandas import GeoSeries
from geopandas.array import GeometryArray
from shapely.geometry import GeometryCollection, MultiPoint, MultiPolygon, box
import pygeos
import shapely.wkb
from geopandas._vectorized import from_wkb, from_wkt
import shapely.geometry

BOUNDS = ("MINX", "MINY", "MAXX", "MAXY")  # "X", "Y")
BOUNDS = list(BOUNDS)
ps.to_dataframe()[[*BOUNDS]].to_numpy().T
df = ps.to_dataframe()
# print(GeoSeries(points))
pollys = GeoSeries(
    pygeos.to_shapely(
        pygeos.box(
            df["MINX"].to_numpy(),
            df["MINY"].to_numpy(),
            df["MAXX"].to_numpy(),
            df["MAXY"].to_numpy(),
        )
    )
)  # .touches(MultiPoint(points))

# ds[["lat", "lon"]].to_dataframe()
# np.array(np.meshgrid(ds.lat, ds.lon)).T.shape
# MultiPoint(np.array(np.meshgrid(ds.lat, ds.lon)).T[:, np.newaxis])
# ds[["lat", "lon"]]
# df.bounds

In [15]:
pollys.contains(points.to_frame().T)
# points.touches(pollys)

/opt/venv/lib/python3.10/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


0        False
1        False
2        False
3        False
4        False
         ...  
76015    False
76016    False
76017    False
76018    False
76019    False
Length: 76020, dtype: bool

In [6]:
nb = GaussianNB()

In [7]:
def split_trin_test(split=0.6):
    ...


Path.cwd() / "data"

# from collections import namedtuple
# Store =

PosixPath('/workspaces/swagpy/notebooks/data')